In [19]:
from pygraphblas import *
from pygraphblas.demo.gviz import draw, draw_op
import pygraphblas.descriptor
import csv
import sys



In [31]:
def load_node(filename):
    
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
        original_ids = [row['id:ID'] for row in reader]
        id_mapping = {}
        for index in range(len(original_ids)):
            id_mapping[original_ids[index]] = index
            
        return original_ids, id_mapping
    
def print_matrix(matrix):
    print(f'Matrix is: {matrix}')
    for row in range(matrix.nrows):
        for col in range(matrix.ncols):
            try:
                value = matrix[row, col]
            except:
                value = ''   
            print('{:4}'.format(value), end = '')
        print()

def load_edge(filename, start_mapping, end_mapping, typ=BOOL, drop_dangling_edges=False):
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
        row_ids = []
        col_ids = []
        values = []
        for row in reader:
            start_id = row['id:START_ID']
            end_id = row['id:END_ID']
            if not drop_dangling_edges or (start_id in start_mapping and end_id in end_mapping):
                row_ids.append(start_mapping[start_id])
                col_ids.append(end_mapping[end_id])
                values.append(1)                
            
        edge_matrix = Matrix.from_lists(
        row_ids,
        col_ids,
        values,
        nrows=len(start_mapping), ncols=len(end_mapping), typ=typ)
        return edge_matrix


In [32]:
route_ids, route_mapping = load_node('trainbenchmark-repair-models-csv/railway-repair-1-Route.csv')
swp_ids, swp_mapping = load_node('trainbenchmark-repair-models-csv/railway-repair-1-SwitchPosition.csv')
sw_ids, sw_mapping = load_node('trainbenchmark-repair-models-csv/railway-repair-1-Switch.csv')
sen_ids, sen_mapping = load_node('trainbenchmark-repair-models-csv/railway-repair-1-Sensor.csv')


In [40]:
follows_mx = load_edge('trainbenchmark-repair-models-csv/railway-repair-1-follows.csv', route_mapping, swp_mapping)
target_mx = load_edge('trainbenchmark-repair-models-csv/railway-repair-1-target.csv', swp_mapping, sw_mapping)
monitored_by_mx = load_edge('trainbenchmark-repair-models-csv/railway-repair-1-monitoredBy.csv', sw_mapping, sen_mapping, drop_dangling_edges=True)
requires_mx = load_edge('trainbenchmark-repair-models-csv/railway-repair-1-requires.csv', route_mapping, sen_mapping)
print_matrix(monitored_by_mx)

Matrix is: <Matrix (25x112 : 98:BOOL)>
   1   1   1   1       1   1                                                                                                                                                                                                                                                                                                                                                                                                                                    
                               1   1                                                                                                                                                                                                                                                                                                                                                                                                                            
                                       1   1   1   1   1   1   

In [ ]:
# follows_mx = Matrix.from_lists(
# [0,1],
# [0,1],    
# [1,1])

# draw(follows_mx)

# target_mx = Matrix.from_lists(
# [0,1],
# [0,0],
# [1,1])

# monitored_by_mx = Matrix.from_lists(
# [0,0],
# [0,2],
# [1,1])

# requires_mx = Matrix.from_lists(
# [0,0,1,1],
# [0,1,0,2],
# [1,1,1,1])


In [43]:
route_to_switch = follows_mx @ target_mx
#print_matrix(route_to_switch)
route_to_sensor = route_to_switch @ monitored_by_mx
#print_matrix(route_to_sensor)
#print_matrix(requires_mx)
route_sensor_violations_query_result = route_to_sensor.eadd(requires_mx, add_op=minus)
#print_matrix(route_sensor_violations_query_result)
x = route_to_sensor.extract_matrix(mask=requires_mx, desc=descriptor.ooco)
print_matrix(x)
#TODO: Result should include -1

Matrix is: <Matrix (5x112 : 12:BOOL)>
                           1                                                                                                                                                                                                                                                                                                                                                                                                                                    
                                                                                                                                                                                                                                                                                                                                                                                                                                                                
                                                               1

In [ ]:
draw(monitored_by_mx

In [ ]:
print_matrix(target_mx)

In [ ]:
print('{:4}'.format(''), end = '')

In [ ]:
target_mx.vals